### Download and make the dataset ready in Kaggle 


In [1]:

# ## uncomment if The zip file of the dataset isn't downloaded,extraced 
# !pip install gdown
# Copy the link. The file ID is the long string of characters between d/ and /view.
#For example, in the URL https://drive.google.com/file/d/1aBcDeFgHiJkLmNoPqRsTuVwXyZ/view?usp=sharing, 
#the file ID is 1aBcDeFgHiJkLmNoPqRsTuVwXyZ
# !mkdir /kaggle/tmp
# !gdown  1pzXpA5Cz0DJmjRsLxlqRNnJq-kOUvojb -O /kaggle/tmp/Labeled_CICMODBUS2023.zip
# !unzip /kaggle/tmp/Labeled_CICMODBUS2023.zip -d /kaggle/working/

# # ## uncomment if the python modules (modbus.py,utils.py ,...) not cloned  and added to the path 

# !git clone https://github.com/hamid-rd/FLBased-ICS-NIDS.git
# import sys
# # Add the repository folder to the Python path
# repo_path = '/kaggle/working/FLBased-ICS-NIDS'
# repo_input_path = '/kaggle/input/training/FLBased-ICS-NIDS'
# dataset_path = '/kaggle/input/training/'

# for path in {repo_path,repo_input_path,dataset_path}:
#     if path not in sys.path:
#         sys.path.append(path)


In [2]:
# To test if every thing is okay (modbus.py class and correct number of founded csv files )
from modbus import ModbusDataset,ModbusFlowStream

# dataset_directory = "/kaggle/working/ModbusDataset" 
# dataset_directory = "/kaggle/input/training/ModbusDataset" 
dataset_directory = "dataset" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

# Don't forget to save version in kaggle (to save outputs written on the disk (/kaggle/working/))  

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

### Unsupervised Autoencoder training  

In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np # For standard deviation calculation
from modbus import ModbusDataset,ModbusFlowStream
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,recall_score
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from utils import load_scalers
import random
from random import SystemRandom
from sklearn.model_selection import train_test_split
import itertools
import torch.nn.init as init
from collections import Counter

def compute_threshold(mse_values,k=1):

    """
    K-SIGMA
    Computes the anomaly detection threshold (for marking sample as Intrusion if the IS was greater )
    based on the mean and standard deviation of Mean Squared Error (MSE) values.
    Formula: thr = mean(MSE) + std(MSE)
    Args:
    mse_values (torch.Tensor or list/np.array): A tensor or list of MSE values

                            obtained from the validation set.
    Returns:
    float: The calculated threshold.
    float: The calculated std.

    """
    if not isinstance(mse_values, torch.Tensor):
        mse_values = torch.tensor(mse_values, dtype=torch.float32)
    if mse_values.numel() == 0:
        return 0.0
    mean_mse = torch.mean(mse_values)
    std_mse = torch.std(mse_values)
    print("-----------mse_loss mean : ",f"{mean_mse.item():.4f}","std:",f"{std_mse.item():.4f}")
    threshold = mean_mse + k*std_mse
    return threshold.item(),std_mse.item()

def vae_loss_function(recon_x, x, mu, logvar,beta=0.05):
    """
    VAE loss function.
    """
    #equivalent to BCE loss
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # print(BCE.item(),KLD.item())
    return (BCE + beta*KLD)

def _init_weights( module):
    ## for one layer apply Xavier Initialization
    if isinstance(module, nn.Linear):
        init.xavier_normal_(module.weight)
        if module.bias is not None:
            init.zeros_(module.bias)
    return module


In [4]:
# dataset_directory = "/kaggle/input/training/ModbusDataset" # change this to the folder contain benign and attack subdirs
# dataset_directory = "dataset" 
dataset_directory = "./ModbusDataset/" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

In [5]:

# AutoEncoder (AE)
class AE(nn.Module):
    """
    Encoder: (76-32-16-4-2)
    Decoder: (2-4-16-32-76)
    """
    def __init__(self,input_dim=76):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon


# Variational AutoEncoder (VAE)
class VAE(nn.Module):
    """
    Encoder: (76-16-4-2 for mu and log_var)
    Decoder: (2-4-16-76)
    return x_recon, mu, logvar
    """
    def __init__(self,input_dim=76):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 4),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(4, 2)
        self.fc_logvar = nn.Linear(4, 2)
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, input_dim),
            nn.Sigmoid()
                    )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar

    
class AAE_Encoder(nn.Module):
    def __init__(self,input_dim=76):
        """
        Encoder(Generator):(76-16-4-2)
        """
        super(AAE_Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 4),
            nn.LeakyReLU(0.2),
            nn.Linear(4, 2))
    def forward(self, x):
        return self.encoder(x)
class AAE_Decoder(nn.Module):
    def __init__(self,input_dim=76):
        super(AAE_Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 16),
            nn.LeakyReLU(),
            nn.Linear(16, input_dim),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.decoder(x)
class AAE_Discriminator(nn.Module):
    def __init__(self):
        super(AAE_Discriminator, self).__init__()
        # corrected to 2-16-4-1
        self.discriminator = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 1), 
            nn.Sigmoid()
        )    
    def forward(self, x):
        return self.discriminator(x)
 
class AdversarialAutoencoder(nn.Module):
    def __init__(self):
        super(AdversarialAutoencoder, self).__init__()
        self.encoder = AAE_Encoder()
        self.decoder = AAE_Decoder()
        self.discriminator = AAE_Discriminator()
    def forward(self, x):
        fake_z = self.encoder(x)
        x_recon = self.decoder(fake_z)
        return fake_z,x_recon


In [6]:
from torch.utils.data import Dataset
class InMemoryDataset(Dataset):
    """A simple dataset that serves data from pre-loaded tensors."""
    def __init__(self, data, labels):
        # This dataset holds references to the data, which should already be in shared memory.
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

### Part a: Centralized learning 

##### You can go from here right to the FL part


In [32]:
def make_dataloader(load_all_chunks=True):

    ## must be called after that the global train_files,val_files and test_files plus loades scalers are initilized
    ## return train, valid and test dataloaders
    if load_all_chunks==True:
        large_chunk_size = modbus.dataset["metadata"]["founded_files_num"]["total_dataset_num"]

        dataset_configs = {
            "train": {"files": train_files},
            "val": {"files": val_files},
            "test": {"files": test_files},
        }
        datasets = {}
        ae_datasets = {}
        dataloaders = {}

        print("Cow Processing datasets...")

        for name, config in dataset_configs.items():
            print(f"  - Creating '{name}' dataset...")
            
            # 1. Create the primary ModbusFlowStream dataset
            datasets[name] = ModbusFlowStream(
                shuffle=False,
                chunk_size=large_chunk_size,
                batch_size=1,
                csv_files=config["files"],
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
                window_size=window_size
            )
            
            # 2. Call __getitem__(0) once to load the entire dataset chunk into memory
            datasets[name].__getitem__(0)

            # 2. Extract the raw tensors
            all_data = datasets[name].current_chunk_data
            all_labels = datasets[name].current_chunk_labels
            
            # all_data.share_memory_()
            # all_labels.share_memory_()
            
            # 4. Create an instance of our SIMPLE dataset using the SHARED tensors.
            shared_dataset = InMemoryDataset(all_data, all_labels)

            # 5. Create the DataLoader from the simple dataset. This will work correctly with workers.
            if name=="train":
                shuffle_samples =True
            else:
                shuffle_samples =False
            # dataloaders[name] = DataLoader(
            #     shared_dataset,
            #     batch_size=64,
            #     shuffle=shuffle_samples,
            #     num_workers=4
            #     persistent_workers=True,
            #     pin_memory=True
            # )
            dataloaders[name] = DataLoader(
                shared_dataset,
                batch_size=64,
                shuffle=shuffle_samples,
                num_workers=0, # You can now use multiple workers effectively.
                persistent_workers=False,
                pin_memory=False
            )
                    
            
            print(f"  - Finished '{name}' dataset.")
        train_dataloader = dataloaders['train']
        val_dataloader = dataloaders['val']
        test_dataloader = dataloaders['test']

    else :
        train_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=True,chunk_size=1,batch_size=64,csv_files=train_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),  batch_size=1,shuffle=False)
        val_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=False,chunk_size=1,batch_size=64,csv_files=val_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),batch_size=1,shuffle=False)
        test_dataloader=DataLoader(ModbusFlowStream(shuffle=False,chunk_size=1,batch_size=64,csv_files=test_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size),
                                    batch_size=1,shuffle=False)
    return train_dataloader,val_dataloader,test_dataloader


In [46]:

def train_eval(model,train_dataloader,val_dataloader,test_dataloader,learning_rates= [5e-6,1e-7,5e-5,1e-5,1e-6],
               weight_decays=[1e-5,1e-4,1e-7],shuffle_files=True,num_epochs=20,
               eval_epoch=4,criterion_method="mse", k_range=[1,3],train_model=True,threshold_1=None):
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model=model.to(device)
    if criterion_method=="bce":
        criterion = nn.BCELoss(reduction='sum').to(device)
        eval_criterion = nn.BCELoss(reduction='none').to(device)
    else: #mse
        criterion = nn.MSELoss(reduction='sum').to(device)
        eval_criterion = nn.MSELoss(reduction='none').to(device)

    for lr, wd in itertools.product(learning_rates, weight_decays):
        if model._get_name()=="AdversarialAutoencoder":
            adversarial_criterion= nn.BCELoss(reduction="sum")
            optimizer_D = optim.Adam(model.discriminator.parameters(), lr=lr, weight_decay=wd)
            optimizer_G =  optim.Adam(list(model.encoder.parameters()) + list(model.decoder.parameters()), lr=lr, weight_decay=wd)
        else:
            AE_optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
            ### new code
            # AE_optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd)

        print(f"\n==================  lr={lr}, wd={wd} ==================")
        if train_model==True:
            model.apply(_init_weights)
        for epoch in range(num_epochs):
            if train_model==True:
                time_1 = time.time()
                model.train()
                train_loss = 0
                ## for AAE
                Discriminator_loss = 0
                for sequences, labels in train_dataloader:
                    sequences=sequences.squeeze().to(device)
                    if labels.sum()!=0:
                        continue
                    if model._get_name()=="AdversarialAutoencoder":
                        # 1) generator loss

                        target_ones= torch.ones(sequences.size(0), 1,device=device,dtype=torch.float)
                        target_zeros= torch.zeros(sequences.size(0), 1,device=device,dtype=torch.float)
                        random_latent = torch.randn(sequences.size(0), 2, device=device)
                        optimizer_G.zero_grad()
                        fake_z,decoded_seq = model(sequences)
                        G_loss = 0.001*adversarial_criterion(model.discriminator(fake_z),target_ones ) + 0.999*criterion(decoded_seq, sequences)
                        G_loss.backward()
                        optimizer_G.step()
                        # 2) discriminator loss
                        optimizer_D.zero_grad()
                        real_loss = adversarial_criterion(model.discriminator(random_latent), target_ones)
                        fake_loss = adversarial_criterion(model.discriminator(fake_z.detach()),  target_zeros)
                        D_loss = 0.5*(real_loss + fake_loss)
                        D_loss.backward()
                        optimizer_D.step()
                        train_loss+=G_loss.item()
                        Discriminator_loss+=D_loss.item()   
                    else:
                        AE_optimizer.zero_grad()
                        if model._get_name()=="AE":
                            recon = model(sequences)
                            loss = criterion(recon, sequences) / sequences.size(0)
                        elif model._get_name()=="VAE" :
                            recon, mu, logvar = model(sequences)
                            loss = vae_loss_function(recon, sequences, mu, logvar) /sequences.size(0)
                        loss.backward()
                        AE_optimizer.step()
                        train_loss += loss.item()
                print(f"Train : time {(time.time()-time_1):.2f} s",
                f"Epoch {epoch+1}")
                if model._get_name()=="AdversarialAutoencoder":
                    print(f"Generator Loss: {train_loss / len(train_dataloader):.4f}",
                        f"Discriminator Loss: {Discriminator_loss / len(train_dataloader):.4f}")
                else:
                    print(f"Train Loss: {train_loss / len(train_dataloader):.4f}")
            # Evaluate part
            if (epoch + 1) % eval_epoch == 0:
                model.eval() 
                all_val_losses = []
                all_val_labels = []
                print(f"--- Running Evaluation for Epoch {epoch+1} lr ={lr} wd {wd} ---")
                with torch.no_grad():
                    for sequences, labels in val_dataloader:
                        sequences = sequences.squeeze().to(device) 
                        if criterion_method=="bce":
                            ## may test features be greater than 1 after scaling 
                            sequences=torch.clamp(sequences, min=0.0, max=1.0)      
                        if model._get_name()=="AE":
                            recon = model(sequences)
                        elif model._get_name()=="VAE" :
                            recon, _, _ = model(sequences)
                        elif model._get_name()=="AdversarialAutoencoder":
                            _,recon= model(sequences)
                        val_loss = eval_criterion(recon, sequences)
                        if val_loss.dim() > 1:
                            val_loss = val_loss
                        else:
                            val_loss = val_loss.unsqueeze(dim=0)
                            labels = labels.unsqueeze(dim=0)
                        val_loss = val_loss.sum(dim=1)
                        all_val_losses.extend(val_loss.cpu().numpy())
                        all_val_labels.extend(labels.flatten().cpu().numpy())
                if(threshold_1 is None):
                    threshold_1,std_mse = compute_threshold(all_val_losses,k=0)
                else:
                    std_mse=0
                    k_range=[0]
                all_val_losses = np.array(all_val_losses).squeeze()  
                all_val_labels = np.array(all_val_labels).squeeze()  
                # If intrusion score > threshold, predict 1 (intrusion), else 0 (benign)
                # For FDR, get True Positives (TP) and False Positives (FP)
                
                predictions = (all_val_losses > threshold_1).astype(int)

                accuracy = accuracy_score(all_val_labels, predictions)
                print(f"Val: Accuracy: {accuracy:.4f}  ")
                
                model.eval() 
                all_test_losses = []
                all_test_labels = []
                with torch.no_grad():
                    for sequences, labels in test_dataloader:
                        sequences = sequences.squeeze().to(device)
                        labels = labels.squeeze().to(device)
                        if criterion_method=="bce":
                            ## may test features be greater than 1 after scaling 
                            sequences=torch.clamp(sequences, min=0.0, max=1.0)
                        if model._get_name()=="AE":
                            recon = model(sequences)
                        elif model._get_name()=="VAE"  :
                            recon, mu, logvar = model(sequences)
                        elif model._get_name()=="AdversarialAutoencoder":
                            _,recon= model(sequences)

                        intrusion_scores = eval_criterion(recon, sequences)
                        if intrusion_scores.dim() > 1:
                            intrusion_scores = intrusion_scores
                        else:
                            intrusion_scores = intrusion_scores.unsqueeze(dim=0)
                            labels = labels.unsqueeze(dim=0)
                        if intrusion_scores.dim()==3:
                            ##GRU : mean of window
                            intrusion_scores = intrusion_scores.mean(dim=1)
                        intrusion_scores = intrusion_scores.sum(dim=1)
                        all_test_losses.extend(intrusion_scores.cpu().numpy())
                        all_test_labels.extend(labels.cpu().numpy())

                all_test_losses = np.array(all_test_losses)
                all_test_labels = np.array(all_test_labels)
                for k in k_range:
                    threshold=threshold_1+k*std_mse
                    print(f" K: {k} K-SIGMA Threshold : ---thr {threshold:.4}")
                    predictions = (all_test_losses > threshold).astype(int)
                    binary_test_labels = (all_test_labels != 0).astype(int)

                    # Find the indices where the prediction was incorrect
                    misclassified_indices = np.where(binary_test_labels != predictions)[0]

                    # Get the original labels for those misclassified instances
                    misclassified_original_labels = all_test_labels[misclassified_indices]

                    # To get a summary count of which labels were misclassified
                    print("Counts of : original binary labels",Counter(binary_test_labels),"predicted binary labels",Counter(predictions))
                    print(f"Counts of  original  labels: {dict(sorted(Counter(all_test_labels).items()))}")
                    print(f"Counts of misclassified original labels: {dict(sorted(Counter(misclassified_original_labels).items()))}")
                    accuracy = accuracy_score(binary_test_labels, predictions)
                    f1 = f1_score(binary_test_labels, predictions, zero_division=0)
                    recall = recall_score(binary_test_labels, predictions,zero_division=0)
                    _, fp, _, tp = confusion_matrix(binary_test_labels, predictions, labels=[0, 1]).ravel()
                    # FDR = FP / (FP + TP) 
                    if (fp + tp) == 0:
                        fdr = 0.0 
                    else:
                        fdr = fp / (fp + tp)
                    print(f"Test : Accuracy: {accuracy:.4f} Recall : {recall:.4f} FDR: {fdr:.4f}  F1-score: {f1:.4f}  ")
                    !mkdir best_center_models -p
                    if train_model:
                        save_path ="best_center_models/"+model._get_name()+"_f1_"+f"{f1:.4f}" +"_recall_"+f"{recall:.4f}" +"_.pth"
                        torch.save(model.state_dict(),save_path)
                        print("model",model._get_name(),"is saved in" ,save_path )


#### Train on network-wdie
#### evaluate on comrpomised-scada IED node ied1b during centralized training.

In [52]:
train_files=[col for col in modbus.dataset["benign_dataset_dir"] if col.find("network-wide")!=-1][:]
test_files= [col for col in modbus.dataset["attack_dataset_dir"]["compromised-scada"] if col.find("ied1b")!=-1]


### missed attack logs files for the day 21 for ied1b which can reduce the accuracy.
test_files.remove(dataset_directory+"attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")
# test_files.remove(dataset_directory+"/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")
SEED=20
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
random.shuffle(train_files)
random.shuffle(test_files)

train_files = train_files[:-3]
val_files = train_files[-3:]
test_files = test_files[:2]
loaded_scalers=load_scalers("fitted_scalers")
print("ied1b comp ied attack ->\n test: ",len(test_files),test_files)
print("----------- Network-wide number of csv files -> \n ----------- train :",len(train_files),train_files,"\n -------- valid:",len(val_files),val_files)

Successfully loaded scalers for 'network-wide'
ied1b comp ied attack ->
 test:  2 ['./ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-7-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-3-labeled.csv']
----------- Network-wide number of csv files -> 
 ----------- train : 16 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-27-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-29-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-20-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-19-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-15-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-14-labeled.csv

In [33]:
def make_dataloader(load_all_chunks=True):

    ## must be called if global train_files,val_files and test_files plus loades scalers are initilized
    ## return train, valid and test dataloaders
    if load_all_chunks:
        large_chunk_size = modbus.dataset["metadata"]["founded_files_num"]["total_dataset_num"]

        dataset_configs = {
            "train": {"files": train_files},
            "val": {"files": val_files},
            "test": {"files": test_files},
        }
        datasets = {}
        ae_datasets = {}
        dataloaders = {}

        print("Cow Processing datasets...")

        for name, config in dataset_configs.items():
            print(f"  - Creating '{name}' dataset...")
            
            # 1. Create the primary ModbusFlowStream dataset
            datasets[name] = ModbusFlowStream(
                shuffle=False,
                chunk_size=large_chunk_size,
                batch_size=1,
                csv_files=config["files"],
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
                window_size=window_size
            )
            
            # 2. Call __getitem__(0) once to load the entire dataset chunk into memory
            datasets[name].__getitem__(0)

            # 2. Extract the raw tensors
            all_data = datasets[name].current_chunk_data
            all_labels = datasets[name].current_chunk_labels
            
            # all_data.share_memory_()
            # all_labels.share_memory_()
            
            # 4. Create an instance of our SIMPLE dataset using the SHARED tensors.
            shared_dataset = InMemoryDataset(all_data, all_labels)

            # 5. Create the DataLoader from the simple dataset. This will work correctly with workers.
            if name=="train":
                shuffle_samples =True
            else:
                shuffle_samples =False
            # dataloaders[name] = DataLoader(
            #     shared_dataset,
            #     batch_size=64,
            #     shuffle=shuffle_samples,
            #     num_workers=4
            #     persistent_workers=True,
            #     pin_memory=True
            # )
            dataloaders[name] = DataLoader(
                shared_dataset,
                batch_size=64,
                shuffle=shuffle_samples,
                num_workers=0, # You can now use multiple workers effectively.
                persistent_workers=False,
                pin_memory=False
            )
                    
            
            print(f"  - Finished '{name}' dataset.")
        train_dataloader = dataloaders['train']
        val_dataloader = dataloaders['val']
        test_dataloader = dataloaders['test']

    else :
        train_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=True,chunk_size=1,batch_size=64,csv_files=train_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),  batch_size=1,shuffle=False)
        val_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=False,chunk_size=1,batch_size=64,csv_files=val_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),batch_size=1,shuffle=False)
        test_dataloader=DataLoader(ModbusFlowStream(shuffle=False,chunk_size=1,batch_size=64,csv_files=test_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size),
                                    batch_size=1,shuffle=False)
    return train_dataloader,val_dataloader,test_dataloader


In [ ]:
train_dataloader,val_dataloader,test_dataloader= make_dataloader(load_all_chunks=False)


In [109]:
t_1=time.time()
for seq,label in train_dataloader:
     pass
print(time.time()-t_1)

33.09274983406067


In [110]:
print(len(train_dataloader),len(val_dataloader),len(test_dataloader))

36920 7193 2174


In [112]:
AE_model = AE(input_dim=76)
train_eval(AE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=2,eval_epoch=2,criterion_method="mse",learning_rates=[1e-2,1e-3,1e-4,1e-5],weight_decays=[1e-4,1e-5],k_range=[1,3])
# --- Running Evaluation for Epoch 6 lr =0.001 wd 1e-05 ---



==================  lr=0.01, wd=0.0001 ==================
Train : time 164.74 s Epoch 1
Train Loss: 0.0511
Train : time 177.71 s Epoch 2
Train Loss: 0.0156
--- Running Evaluation for Epoch 2 lr =0.01 wd 0.0001 ---
-----------mse_loss mean :  0.0093 std: 0.1554
Val: Accuracy: 0.9101  
 K: 1 K-SIGMA Threshold : ---thr 0.1647
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 102696, 1: 36354})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 94, 1: 5260, 2: 14, 3: 15, 4: 4, 5: 1, 6: 10}
Test : Accuracy: 0.9612 Recall : 0.8724 FDR: 0.0026  F1-score: 0.9307  
model AE is saved in best_models/AE_f1_0.93_recall_0.87_.pth
 K: 3 K-SIGMA Threshold : ---thr 0.4756
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 103018, 1: 36032})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 

In [113]:
VAE_model = VAE(input_dim=76)
train_eval(VAE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=2,eval_epoch=2,criterion_method="mse",learning_rates=[1e-2,1e-3,1e-4,1e-5],weight_decays=[1e-3,1e-4],k_range=[1,3])



==================  lr=0.01, wd=0.001 ==================
Train : time 165.62 s Epoch 1
Train Loss: 0.2089
Train : time 184.23 s Epoch 2
Train Loss: 0.1849
--- Running Evaluation for Epoch 2 lr =0.01 wd 0.001 ---
-----------mse_loss mean :  0.0732 std: 0.4997
Val: Accuracy: 0.8659  
 K: 1 K-SIGMA Threshold : ---thr 0.5729
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 102985, 1: 36065})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 102, 1: 5530, 2: 14, 3: 16, 4: 4, 5: 1, 6: 10, 7: 26}
Test : Accuracy: 0.9590 Recall : 0.8652 FDR: 0.0028  F1-score: 0.9265  
model VAE is saved in best_models/VAE_f1_0.93_recall_0.87_.pth
 K: 3 K-SIGMA Threshold : ---thr 1.572
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 103032, 1: 36018})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 

In [114]:
AAE_model = AdversarialAutoencoder()
train_eval(AAE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=2,eval_epoch=2,criterion_method="mse",learning_rates=[1e-2,1e-3,1e-4,1e-5],weight_decays=[1e-3,1e-4],k_range=[1,3])



==================  lr=0.01, wd=0.001 ==================
Train : time 260.97 s Epoch 1
Generator Loss: 2.7325 Discriminator Loss: 25.2167
Train : time 260.60 s Epoch 2
Generator Loss: 0.8408 Discriminator Loss: 26.6410
--- Running Evaluation for Epoch 2 lr =0.01 wd 0.001 ---
-----------mse_loss mean :  0.0046 std: 0.0674
Val: Accuracy: 0.9078  
 K: 1 K-SIGMA Threshold : ---thr 0.07198
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 102696, 1: 36354})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 94, 1: 5260, 2: 14, 3: 15, 4: 4, 5: 1, 6: 10}
Test : Accuracy: 0.9612 Recall : 0.8724 FDR: 0.0026  F1-score: 0.9307  
model AdversarialAutoencoder is saved in best_models/AdversarialAutoencoder_f1_0.93_recall_0.87_.pth
 K: 3 K-SIGMA Threshold : ---thr 0.2068
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counte

K=3 is better in AAE model

In [118]:
#lr=0.001, wd=1e-05 
AE_model = AE(input_dim=76)
train_eval(AE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=20,eval_epoch=4,criterion_method="mse",learning_rates=[1e-3],weight_decays=[1e-5],k_range=[1])



==================  lr=0.001, wd=1e-05 ==================
Train : time 157.69 s Epoch 1
Train Loss: 0.3925
Train : time 153.40 s Epoch 2
Train Loss: 0.0358
Train : time 152.62 s Epoch 3
Train Loss: 0.0248
Train : time 153.83 s Epoch 4
Train Loss: 0.0219
--- Running Evaluation for Epoch 4 lr =0.001 wd 1e-05 ---
-----------mse_loss mean :  0.0232 std: 0.3594
Val: Accuracy: 0.9913  
 K: 1 K-SIGMA Threshold : ---thr 0.3826
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 103017, 1: 36033})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 93, 1: 5552, 2: 14, 3: 15, 4: 4, 5: 1, 6: 10, 7: 28}
Test : Accuracy: 0.9589 Recall : 0.8647 FDR: 0.0026  F1-score: 0.9263  
model AE is saved in best_center_models/AE_f1_0.9263_recall_0.8647_.pth
Train : time 152.49 s Epoch 5
Train Loss: 0.0216
Train : time 157.49 s Epoch 6
Train Loss: 0.0214
Train : time 160.24 s Ep

In [119]:
#==================  lr=0.0001, wd=0.0001 ==================

VAE_model = VAE(input_dim=76)
train_eval(VAE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=20,eval_epoch=4,criterion_method="mse",learning_rates=[1e-4],weight_decays=[1e-4,],k_range=[1])



==================  lr=0.0001, wd=0.0001 ==================
Train : time 195.29 s Epoch 1
Train Loss: 0.7271
Train : time 188.97 s Epoch 2
Train Loss: 0.1814
Train : time 196.54 s Epoch 3
Train Loss: 0.1734
Train : time 204.82 s Epoch 4
Train Loss: 0.1685
--- Running Evaluation for Epoch 4 lr =0.0001 wd 0.0001 ---
-----------mse_loss mean :  0.0537 std: 0.2896
Val: Accuracy: 0.8444  
 K: 1 K-SIGMA Threshold : ---thr 0.3433
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 102169, 1: 36881})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 711, 1: 5348, 2: 14, 3: 14, 4: 4, 5: 1, 6: 10, 7: 3}
Test : Accuracy: 0.9561 Recall : 0.8702 FDR: 0.0193  F1-score: 0.9222  
model VAE is saved in best_center_models/VAE_f1_0.9222_recall_0.8702_.pth
Train : time 200.85 s Epoch 5
Train Loss: 0.1655
Train : time 177.86 s Epoch 6
Train Loss: 0.1617
Train : time 162.7

In [120]:
# ==================  lr=0.01, wd=0.0001 ==================

AAE_model = AdversarialAutoencoder()
train_eval(AAE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=20,eval_epoch=4,criterion_method="mse",learning_rates=[1e-2],weight_decays=[1e-4])



==================  lr=0.01, wd=0.0001 ==================
Train : time 258.46 s Epoch 1
Generator Loss: 2.5820 Discriminator Loss: 23.0968
Train : time 263.93 s Epoch 2
Generator Loss: 1.0836 Discriminator Loss: 26.7439
Train : time 260.17 s Epoch 3
Generator Loss: 0.5757 Discriminator Loss: 32.0621
Train : time 259.62 s Epoch 4
Generator Loss: 1.3244 Discriminator Loss: 27.5940
--- Running Evaluation for Epoch 4 lr =0.01 wd 0.0001 ---
-----------mse_loss mean :  0.0067 std: 0.0696
Val: Accuracy: 0.8607  
 K: 1 K-SIGMA Threshold : ---thr 0.07626
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 102696, 1: 36354})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 94, 1: 5260, 2: 14, 3: 15, 4: 4, 5: 1, 6: 10}
Test : Accuracy: 0.9612 Recall : 0.8724 FDR: 0.0026  F1-score: 0.9307  
model AdversarialAutoencoder is saved in best_center_models/AdversarialA

K=3 is better for AAE than K=1

#### Evaluate pre-trained autoencoders  on the compromised-ied and compromised scada scenarios 

No exact labeling for the comp ied scenario results in low performance 

In [21]:
Trained_AE_model=AE(input_dim=76)
Trained_AE_model.load_state_dict(torch.load("./best_center_models/AE_f1_0.9268_recall_0.8655_.pth"))
Trained_VAE_model=VAE(input_dim=76)
Trained_VAE_model.load_state_dict(torch.load("./best_center_models/VAE_f1_0.9224_recall_0.8736_.pth"))
Trained_AAE_model=AdversarialAutoencoder()
Trained_AAE_model.load_state_dict(torch.load("./best_center_models/AdversarialAutoencoder_f1_0.9317_recall_0.8724_.pth"))

<All keys matched successfully>

In [22]:
dataset_directory = "./ModbusDataset/" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()


 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

In [34]:
### test on total attack captures
test_files=[col for col in modbus.dataset["attack_dataset_dir"]["external"] if col.find("network-wide")!=-1]
test_files.extend( [col for col in modbus.dataset["attack_dataset_dir"]["compromised-ied"] if col.find("trust-scada-hmi")!=-1])
test_files.extend([col for col in modbus.dataset["attack_dataset_dir"]["compromised-scada"] if col.find("substation-wide-capture")!=-1])
print("test_files length",len(test_files))
print("benign valid files:",len(val_files))

val_dataloader=DataLoader(ModbusFlowStream(
            shuffle=False,
            chunk_size=1,
            batch_size=64,
            csv_files=val_files,
            scalers=loaded_scalers['network-wide']['min_max_scalers'],
        ),batch_size=1,shuffle=False)
test_dataloader=DataLoader(ModbusFlowStream(
            shuffle=False,
            chunk_size=1,
            batch_size=64,
            csv_files=test_files,
            scalers=loaded_scalers['network-wide']['min_max_scalers'],
        ),batch_size=1,shuffle=False)
for trained_model in {Trained_AE_model,Trained_VAE_model,Trained_AAE_model}:
    print("*"*10,trained_model._get_name(),10*"*")
    train_eval(trained_model,None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0])
    

test_files length 40
benign valid files: 3
********** AE **********

==================  lr=0, wd=0 ==================
--- Running Evaluation for Epoch 1 lr =0 wd 0 ---
-----------mse_loss mean :  0.0176 std: 0.2892
Val: Accuracy: 0.9933  
 K: 1 K-SIGMA Threshold : ---thr 0.3067
Counts of : original binary labels Counter({0: 4811517, 1: 396525}) predicted binary labels Counter({0: 4694342, 1: 513700})
Counts of  original  labels: {0: 4811517, 1: 393454, 2: 611, 3: 507, 4: 505, 5: 510, 6: 425, 7: 470, 8: 43}
Counts of misclassified original labels: {0: 168467, 1: 50373, 2: 131, 3: 188, 4: 104, 5: 2, 6: 102, 7: 367, 8: 25}
Test : Accuracy: 0.9578 Recall : 0.8706 FDR: 0.3279  F1-score: 0.7586  
 K: 3 K-SIGMA Threshold : ---thr 0.8851
Counts of : original binary labels Counter({0: 4811517, 1: 396525}) predicted binary labels Counter({0: 4874000, 1: 334042})
Counts of  original  labels: {0: 4811517, 1: 393454, 2: 611, 3: 507, 4: 505, 5: 510, 6: 425, 7: 470, 8: 43}
Counts of misclassified or

In [35]:
for scenario in {"compromised-ied","external","compromised-scada"}:
    if scenario=="compromised-scada":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("ied1b")!=-1]
        ### missed attack logs for the day 21 for ied1b which can reduce the accuracy.
        test_files.remove(dataset_directory+"attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")    
    elif scenario=="compromised-ied":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("trust-scada-hmi")!=-1]
    else:
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("network-wide")!=-1]        

    print("----------- benign valid files:",len(val_files),val_files)
    print(f"----------{scenario} attack  test files : ",len(test_files),test_files)
    val_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=1,
                batch_size=64,
                csv_files=val_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    test_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=1,
                batch_size=64,
                csv_files=test_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    for trained_model in {Trained_AE_model,Trained_VAE_model,Trained_AAE_model}:
        print("*"*10,trained_model._get_name(),10*"*")
        train_eval(trained_model,None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0])
        

scenario : compromised-ied
----------- benign valid files: 3 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-26-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-23-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-21-labeled.csv']
----------compromised-ied attack  test files :  20 ['./ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-2-labeled.csv', './ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-10-labeled.csv', './ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-19-labeled.csv', './ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-17-labeled.csv', './ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/read

In [36]:

## TEST RESULTS WITHOUT IED1B TRAFFIC
modbus = ModbusDataset(dataset_directory,"ready")

for scenario in {"external","compromised-scada"}:

    print(scenario)
    test_files=modbus.dataset["attack_dataset_dir"][scenario]
    nodes_to_exclude = [
    "ied1b",
    "network-wide",
    "substation-wide-capture",
    "scada-hmi-network-capture",
    "trust-scada-hmi",
    "central-agent",
    "ied4c"
    ]
    exclude = [col for col in test_files if any(node in col for node in nodes_to_exclude)]
    [test_files.remove(c) for c in exclude if c in test_files]
    print("exclude",len(exclude),exclude)
    print("filtered test files without ied1b",len(test_files),test_files)
    for test_file in test_files: 
        print(test_file)
        val_dataloader=DataLoader(ModbusFlowStream(
                    shuffle=False,
                    chunk_size=1,
                    batch_size=64,
                    csv_files=val_files,
                    scalers=loaded_scalers['network-wide']['min_max_scalers'],
                ),batch_size=1,shuffle=False)
        test_dataloader=DataLoader(ModbusFlowStream(
                    shuffle=False,
                    chunk_size=1,
                    batch_size=64,
                    csv_files=[test_file],
                    scalers=loaded_scalers['network-wide']['min_max_scalers'],
                ),batch_size=1,shuffle=False)
        for trained_model in {Trained_AE_model,Trained_VAE_model,Trained_AAE_model}:
            print("*"*10,trained_model._get_name(),10*"*")
            train_eval(trained_model,None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0])
            

external
exclude 6 ['./ModbusDataset/attack/external/network-wide/ready/network-wide-normal-1-labeled.csv', './ModbusDataset/attack/external/network-wide/ready/network-wide-normal-0-labeled.csv', './ModbusDataset/attack/external/ied4c/ied4c-network-capture/ready/veth8bc3408-0-labeled.csv', './ModbusDataset/attack/external/ied1b/ied1b-network-capture/ready/vethd9e14c0-0-labeled.csv', './ModbusDataset/attack/external/scada-hmi/scada-hmi-network-capture/ready/veth5bbeaa2-0-labeled.csv', './ModbusDataset/attack/external/central-agent/central-agent-network-capture/ready/veth460b141-0-labeled.csv']
filtered test files without ied1b 2 ['./ModbusDataset/attack/external/external-attacker/external-attacker-network-capture/ready/veth665f3cf-0-labeled.csv', './ModbusDataset/attack/external/ied1a/ied1a-network-capture/ready/veth4edc015-0-labeled.csv']
./ModbusDataset/attack/external/external-attacker/external-attacker-network-capture/ready/veth665f3cf-0-labeled.csv
********** AE **********

=======

### Part b: Federated learning 
####  non iid distribution of dataset (ip\node based)

In [7]:
# ==============================================================================
# 1. SETUP: INSTALL LIBRARIES AND IMPORT DEPENDENCIES
# ==============================================================================
# In a Kaggle notebook, run this cell first to install the necessary libraries.
# !pip install -q flwr[simulation] torch torchvision pandas scikit-learn matplotlib seaborn


In [7]:

from collections import OrderedDict
from typing import Dict, List, Tuple, Optional , Union
import os 
import flwr as fl
import ray
from flwr.common import FitRes, Scalar,Context, ndarrays_to_parameters, parameters_to_ndarrays
from flwr.server.client_proxy import ClientProxy

import random

# Suppress warning messages for a cleaner output
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
# Set a seed for reproducibility
SEED = 20
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#global device
dataset_directory = "./ModbusDataset/" 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")
modbus = ModbusDataset(dataset_directory,"ready")


Training on cuda:0


In [8]:

# ==============================================================================
#  FEDERATED LEARNING CLIENT: FlowerClient
# ==============================================================================
class FlowerClient(fl.client.NumPyClient):
    """Flower client for training."""
    def __init__(self, cid, model, trainloader,valloader):
        self.cid = cid
        self.model = model
        self.train_dataloader = trainloader
        self.val_dataloader = valloader
    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        model =self.model
        lr = cfg.LEARNING_RATE
        wd= cfg.WEIGHT_DECAY
        
        criterion = nn.MSELoss(reduction='sum').to(DEVICE)
        if model._get_name()=="AdversarialAutoencoder":
            adversarial_criterion= nn.BCELoss(reduction="sum")
            optimizer_D = optim.Adam(model.discriminator.parameters(), lr=lr, weight_decay=wd)
            optimizer_G =  optim.Adam(list(model.encoder.parameters()) + list(model.decoder.parameters()), lr=lr, weight_decay=wd)
        else:
            AE_optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=wd)

        if cfg.STRATEGY == "FED_PROX":
            global_params_dict = {
                k: torch.tensor(v, device=DEVICE) 
                for k, v in zip(self.model.state_dict().keys(), parameters)
            }

        for epoch in range(cfg.LOCAL_EPOCHS):
            time_1 = time.time()
            model.train()
            train_loss = 0
            ## for AAE
            Discriminator_loss = 0
            for sequences, _ in self.train_dataloader:
                sequences=sequences.squeeze().to(DEVICE)
                if model._get_name()=="AdversarialAutoencoder":
                    target_ones= torch.ones(sequences.size(0), 1,device=DEVICE,dtype=torch.float)
                    target_zeros= torch.zeros(sequences.size(0), 1,device=DEVICE,dtype=torch.float)
                    random_latent = torch.randn(sequences.size(0), 2, device=DEVICE)
                    optimizer_G.zero_grad()
                    fake_z,decoded_seq = model(sequences)
                    G_loss = 0.001*adversarial_criterion(model.discriminator(fake_z),target_ones ) + 0.999*criterion(decoded_seq, sequences)
                    if cfg.STRATEGY == "FED_PROX":
                        proximal_term_G = 0.0
                        # Proximal term for ENCODER
                        for name, local_param in model.encoder.named_parameters():
                            global_param = global_params_dict['encoder.' + name]
                            proximal_term_G += torch.pow((local_param - global_param).norm(2), 2)
                        # Proximal term for DECODER
                        for name, local_param in model.decoder.named_parameters():
                            global_param = global_params_dict['decoder.' + name]
                            proximal_term_G += torch.pow((local_param - global_param).norm(2), 2)
                        
                        G_loss += (cfg.PROXIMAL_MU / 2) * proximal_term_G
                    G_loss.backward()
                    optimizer_G.step()
                    # 2) discriminator loss
                    optimizer_D.zero_grad()
                    real_loss = adversarial_criterion(model.discriminator(random_latent), target_ones)
                    fake_loss = adversarial_criterion(model.discriminator(fake_z.detach()),  target_zeros)
                    D_loss =  0.5*(real_loss + fake_loss)
                    if cfg.STRATEGY == "FED_PROX":
                        proximal_term_D = 0.0
                        # Proximal term for DISCRIMINATOR
                        for name, local_param in model.discriminator.named_parameters():
                            global_param = global_params_dict['discriminator.' + name]
                            proximal_term_D += torch.pow((local_param - global_param).norm(2), 2)
                        D_loss += (cfg.PROXIMAL_MU / 2) * proximal_term_D
            
                    D_loss.backward()
                    optimizer_D.step()
                    train_loss+=G_loss.item()
                    Discriminator_loss+=D_loss.item()   
                else:
                    AE_optimizer.zero_grad()
                    if model._get_name()=="AE":
                        recon = model(sequences)
                        loss = criterion(recon, sequences) / sequences.size(0)
                    elif model._get_name()=="VAE" :
                        recon, mu, logvar = model(sequences)
                        loss = vae_loss_function(recon, sequences, mu, logvar) /sequences.size(0)
                    
                    if cfg.STRATEGY == "FED_PROX":
                        proximal_term = 0.0
                        for name, local_param in model.encoder.named_parameters():
                            global_param = global_params_dict['encoder.' + name]
                            proximal_term += torch.pow((local_param - global_param).norm(2), 2)
                        # Proximal term for DECODER
                        for name, local_param in model.decoder.named_parameters():
                            global_param = global_params_dict['decoder.' + name]
                            proximal_term += torch.pow((local_param - global_param).norm(2), 2)
                        loss+= (cfg.PROXIMAL_MU / 2) *proximal_term
                    loss.backward()
                    AE_optimizer.step()
                    train_loss += loss.item()
            print(f"Train : time {(time.time()-time_1):.2f} s",
            f"Epoch {epoch+1}")
            num_samples=len(self.train_dataloader)
            if model._get_name()=="AdversarialAutoencoder":
                print(f"Generator Loss: {train_loss / num_samples:.4f}",
                    f"Discriminator Loss: {Discriminator_loss / num_samples:.4f}")
            else:
                print(f"Train Loss: {train_loss / num_samples:.4f}")
        local_threshold,len_val_samples=self._calculate_threshold()
        metrics = {"threshold": local_threshold,"len": len_val_samples}
        return self.get_parameters(config={}), num_samples, metrics

    def _calculate_threshold(self):
        model = self.model
        model.to(DEVICE)
        model.eval()
        eval_criterion = nn.MSELoss(reduction='none').to(DEVICE)

        # Evaluate part
        all_val_losses = []
        all_val_labels = []
        with torch.no_grad():
            for sequences, labels in self.val_dataloader:
                sequences = sequences.squeeze().to(DEVICE) 
                if model._get_name()=="AE":
                    recon = model(sequences)
                elif model._get_name()=="VAE" :
                    recon, _, _ = model(sequences)
                elif model._get_name()=="AdversarialAutoencoder":
                    _,recon= model(sequences)
                val_loss = eval_criterion(recon, sequences)
                if val_loss.dim() > 1:
                    val_loss = val_loss
                else:
                    val_loss = val_loss.unsqueeze(dim=0)
                    labels = labels.unsqueeze(dim=0)
                val_loss = val_loss.sum(dim=1)
                all_val_losses.extend(val_loss.cpu().numpy())
                all_val_labels.extend(labels.flatten().cpu().numpy())     
        threshold_1,std_mse = compute_threshold(all_val_losses,k=1)
        all_val_losses = np.array(all_val_losses).squeeze()  
        all_val_labels = np.array(all_val_labels).squeeze()  
        # If intrusion score > threshold, predict 1 (intrusion), else 0 (benign)
        # For FDR, get True Positives (TP) and False Positives (FP)
        
        predictions = (all_val_losses > threshold_1).astype(int)
        accuracy = accuracy_score(all_val_labels, predictions)
        print(f"Val: Accuracy: {accuracy:.4f}  ")
        return threshold_1,len(all_val_losses)
    def evaluate(self, parameters, config):
        #focuses on server-side evaluation, so we can keep this simple
        return 0.0, 0, {}


In [9]:
class FedAnomalyStrategy(fl.server.strategy.FedAvg):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.aggregated_threshold: Optional[float] = None

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[fl.common.Parameters], Dict[str, Scalar]]:
        
        aggregated_parameters, _ = super().aggregate_fit(server_round, results, failures)
        candidates = [
            (res.metrics["threshold"],res.metrics["len"]) for _, res in results if "threshold" in res.metrics
        ]
        if candidates:
            threholds=np.array([pair[0] for pair in candidates],dtype=float)
            weights=np.array([pair[1] for pair in candidates],dtype=int)

            self.aggregated_threshold = np.average(threholds,weights=weights)
            print(f"Round {server_round}: Aggregated threshold = {self.aggregated_threshold:.4f}")
        else:
            print("Warning: No thresholds received from clients.")
        return aggregated_parameters, {}

In [10]:

# ==============================================================================
#  SERVER-SIDE LOGIC AND SIMULATION START
# ==============================================================================

def client_function(context:Context ) -> FlowerClient:
    client_id = int(context.node_config["partition-id"])
    trainloader = load_data_from_id(client_id,"client",chunk_size=1)
    valloader = load_data_from_id(client_id,"server",chunk_size=1)
    model = get_model().to(DEVICE)
    return FlowerClient(client_id, model, trainloader,valloader).to_client()

def make_client_fn_with_cache(chunk_size=10000):
    print("...")
    dataloader_cache: Dict[str, DataLoader] = {}
    def client_fn(context:Context ) -> FlowerClient:
        client_id = int(context.node_config["partition-id"])
        if client_id not in dataloader_cache:
            # If not, create it once and store it in the cache
            print(f"Round 1: Loading and caching data for client {client_id}...")
            dataloader_cache[client_id] = load_data_from_id(client_id,"client",chunk_size=chunk_size)
        else:
            print(f"Reusing cached dataloader for client {client_id}...")
        trainloader = dataloader_cache[client_id]
        model = get_model().to(DEVICE)
        return FlowerClient(client_id, model, trainloader).to_client()
    return client_fn


def get_evaluate_fn(model, test_dataloader, strategy: FedAnomalyStrategy):
    """Return an evaluation function for server-side evaluation with caching """
    eval_criterion = nn.MSELoss(reduction='none').to(DEVICE)
    best_f1=0
    best_recall=0
    def evaluate(
        server_round: int,
        parameters: fl.common.NDArrays,
        config: Dict[str, fl.common.Scalar],
        train_model=True
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
        nonlocal best_f1,best_recall
        params_dict = zip(model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        model.load_state_dict(state_dict, strict=True)
        model.to(DEVICE)
        model.eval()
        all_test_losses = []
        all_test_labels = []
        temp_best_recall =best_recall
        temp_best_f1 =best_f1
        with torch.no_grad():
            for sequences, labels in test_dataloader:
                sequences = sequences.squeeze().to(DEVICE)
                labels = labels.squeeze().to(DEVICE)
                if model._get_name()=="AE":
                    recon = model(sequences)
                elif model._get_name()=="VAE" :
                    recon, mu, logvar = model(sequences)
                elif model._get_name()=="AdversarialAutoencoder":
                    _,recon= model(sequences)

                intrusion_scores = eval_criterion(recon, sequences)
                if intrusion_scores.dim() > 1:
                    intrusion_scores = intrusion_scores
                else:
                    intrusion_scores = intrusion_scores.unsqueeze(dim=0)
                    labels = labels.unsqueeze(dim=0)
                if intrusion_scores.dim()==3:
                    ##GRU : mean of window
                    intrusion_scores = intrusion_scores.mean(dim=1)
                intrusion_scores = intrusion_scores.sum(dim=1)
                all_test_losses.extend(intrusion_scores.cpu().numpy())
                all_test_labels.extend(labels.cpu().numpy())

        all_test_losses = np.array(all_test_losses)
        all_test_labels = np.array(all_test_labels)
        if strategy.aggregated_threshold is None:
            # Threshold not available yet (e.g., round 0)
            threshold=0
        else:
            threshold = strategy.aggregated_threshold
        test_result = {}

        predictions = (all_test_losses > threshold).astype(int)
        binary_test_labels = (all_test_labels != 0).astype(int)

        # Find the indices where the prediction was incorrect
        misclassified_indices = np.where(binary_test_labels != predictions)[0]

        # Get the original labels for those misclassified instances
        misclassified_original_labels = all_test_labels[misclassified_indices]

        # To get a summary count of which labels were misclassified
        print("Counts of : original binary labels",Counter(binary_test_labels),"predicted binary labels",Counter(predictions))
        print(f"Counts of  original  labels: {dict(sorted(Counter(all_test_labels).items()))}")
        print(f"Counts of misclassified original labels: {dict(sorted(Counter(misclassified_original_labels).items()))}")
        accuracy = accuracy_score(binary_test_labels, predictions)
        f1 = f1_score(binary_test_labels, predictions, zero_division=0)
        recall = recall_score(binary_test_labels, predictions,zero_division=0)
        _, fp, _, tp = confusion_matrix(binary_test_labels, predictions, labels=[0, 1]).ravel()
        # FDR = FP / (FP + TP) 
        if (fp + tp) == 0:
            fdr = 0.0 
        else:
            fdr = fp / (fp + tp)
        test_result[0] = f"threshold={threshold:.4f} ,Test : Accuracy: {accuracy:.4f} Recall : {recall:.4f} FDR: {fdr:.4f}  F1-score: {f1:.4f} "
        print(test_result)
        !mkdir fed_best_models -p
        if f1>best_f1 :
            best_f1=f1
        if recall>best_recall:
            best_recall=recall
        if ((best_recall>temp_best_recall or best_f1 > temp_best_f1) and not(strategy.aggregated_threshold is None)):
            if train_model:
                save_path ="fed_best_models/"+cfg.STRATEGY+"_"+model._get_name()+"_f1_"+f"{best_f1:.2f}" +"_recall_"+f"{best_recall:.2f}" +"_.pth"
                torch.save(model.state_dict(),save_path)
                print("model",model._get_name(),"is saved in" ,save_path )
        return np.sum(all_test_losses)/len(all_test_losses),test_result

    return evaluate


In [11]:

def get_initial_parameters(model_name: str):
    """
    Initializes the model weights using Xavier uniform distribution
    and returns them as a Flower Parameters object.
    """
    
    temp_model = get_model()
    for param in temp_model.parameters():
        if param.dim() > 1:
            nn.init.xavier_uniform_(param)
            
    ndarrays = [val.cpu().numpy() for _, val in temp_model.state_dict().items()]
    return ndarrays_to_parameters(ndarrays)


def load_data_from_id(id: int, node = "client" ,chunk_size=10000):
    """Loads the data for a specific training client."""
    if node == "client":
        file_list = TRAIN_CLIENT_DATA_MAPPING[id]
        shuffle=cfg.SHUFFLE_FILES
    else: # server
        file_list = SERVER_EVALUATION_DATA_MAPPING[id]
        shuffle = False
    ## means load all chunks of data in memory at once 
    if chunk_size==10000:
        train_loader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=chunk_size,
                batch_size=1 ,
                csv_files=file_list,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=64,shuffle=shuffle)
    else :
        train_loader=DataLoader(ModbusFlowStream(
                shuffle=shuffle,
                chunk_size=chunk_size,
                batch_size=cfg.BATCH_SIZE ,
                csv_files=file_list,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    return train_loader

def get_model():
    """Returns the model specified in the config."""
    if cfg.MODEL_NAME == "VAE":
        return VAE(input_dim=cfg.INPUT_DIM)
    elif cfg.MODEL_NAME == "AE":
        return AE(input_dim=cfg.INPUT_DIM)
    elif cfg.MODEL_NAME =="AAE":
        return AdversarialAutoencoder()#76
    else:
        raise ValueError(f"Unknown model name: {cfg.MODEL_NAME}. Choose 'AE' or 'VAE' or 'AAE'.")

def set_server_strategy():
    if cfg.STRATEGY == "FED_PROX":
        print(f"Using FedProx strategy with {cfg.MODEL_NAME} model.")
    else:
        print(f"Using FedAvg strategy with {cfg.MODEL_NAME} model.")
    strategy = FedAnomalyStrategy(
        fraction_fit=1.0,
        fraction_evaluate=0.0,
        min_fit_clients=cfg.NUM_TRAIN_CLIENTS,
        min_available_clients=cfg.NUM_TRAIN_CLIENTS,
        min_evaluate_clients=0,
        initial_parameters=get_initial_parameters(cfg.MODEL_NAME)
    )
    model = get_model().to(DEVICE)
    testloader = load_data_from_id(-1,"server",chunk_size=1)
    evaluate_function = get_evaluate_fn(model, testloader, strategy)
    strategy.evaluate_fn=evaluate_function
    return strategy


#### test on compromised scada attack

In [ ]:

# # ==============================================================================
# #  DATA Distribution
# # ==============================================================================


SEED=20

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

network_train_files=[col for col in modbus.dataset["benign_dataset_dir"] if col.find("network-wide")!=-1][:]
### just for tune hyperparameters
test_files= [col for col in modbus.dataset["attack_dataset_dir"]["compromised-scada"] if col.find("ied1b")!=-1]
### missed attack logs files for the day 21 for ied1b which can reduce the accuracy.
test_files.remove(dataset_directory+"attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")

random.shuffle(network_train_files)
random.shuffle(test_files)

num_splits = 4
train_files = list(np.array_split(network_train_files, num_splits))

SERVER_EVALUATION_DATA_MAPPING = [list(traffic_file[-1:]) for traffic_file in train_files ]
SERVER_EVALUATION_DATA_MAPPING.extend([test_files[:2]])
TRAIN_CLIENT_DATA_MAPPING = [list(traffic_file[:-1]) for traffic_file in train_files ]

for i in range(num_splits):
    print("node_",i+1,"train:",len(TRAIN_CLIENT_DATA_MAPPING[i]),TRAIN_CLIENT_DATA_MAPPING[i])
    print("node_",i+1,"val:",len(SERVER_EVALUATION_DATA_MAPPING[i]),SERVER_EVALUATION_DATA_MAPPING[i])

print("test",len(SERVER_EVALUATION_DATA_MAPPING[-1]),SERVER_EVALUATION_DATA_MAPPING[-1])


node_ 1 train: 4 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-27-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-29-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-20-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-19-labeled.csv']
node_ 1 val: 1 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-15-labeled.csv']
node_ 2 train: 4 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-14-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-28-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-31-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-16-labeled.

In [13]:

# ==============================================================================
#  CONFIGURATION: TWEAK  FEDERATED LEARNING EXPERIMENT
# ==============================================================================
class Config:
    """Global configuration class for the federated learning experiment."""
    # --- FL Parameters ---
    NUM_TRAIN_CLIENTS = num_splits
    NUM_ROUNDS = 10
    LOCAL_EPOCHS = 2
    BATCH_SIZE = 64
    LEARNING_RATE = 1e-3
    WEIGHT_DECAY = 1e-5
    
    # --- Strategy Selection ---
    # Choose from "FED_AVG", "FED_PROX"
    STRATEGY = "FED_AVG" 
    PROXIMAL_MU = 1e-2 # Proximal term for FedProx
    # --- Model Selection ---
    # Choose from "AE" (Autoencoder) or "VAE" (Variational Autoencoder) or "AdverserialAutoencoder"
    MODEL_NAME = "AE"
    INPUT_DIM = 76
    # --- Anomaly Detection ---
    SHUFFLE_FILES=  True
# Instantiate the configuration
cfg = Config()

loaded_scalers = load_scalers("fitted_scalers")




Successfully loaded scalers for 'network-wide'


In [16]:
# ## cache all client data in memory
# client_fn=make_client_fn_with_cache()

# for i in range(3):
#     context= Context(0,i,{},0,0)
#     context.node_config["partition-id"]=i
#     client_fn(context)


In [14]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message=".*DEPRECATED FEATURE: flwr.simulation.start_simulation.*")
warnings.filterwarnings("ignore", category=DeprecationWarning)

strategy=set_server_strategy()


history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

Using FedAvg strategy with AE model.


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout
2025-07-28 00:27:34,233	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'object_store_memory': 1898058547.0, 'memory': 3796117095.0, 'GPU': 1.0, 'node:127.0.1.1': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower V

Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({1: 139050})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 97486}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}


INFO :      initial parameters (loss, other metrics): 17.23868932038835, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


(ClientAppActor pid=2111) Train : time 40.63 s Epoch 1
(ClientAppActor pid=2111) Train Loss: 0.3049
(ClientAppActor pid=2111) Train : time 36.87 s Epoch 2
(ClientAppActor pid=2111) Train Loss: 0.1317
(ClientAppActor pid=2111) -----------mse_loss mean :  0.1062 std: 0.4379
(ClientAppActor pid=2111) Val: Accuracy: 0.9490  
(ClientAppActor pid=2111) Train : time 33.61 s Epoch 1
(ClientAppActor pid=2111) Train Loss: 0.3385
(ClientAppActor pid=2111) Train : time 32.24 s Epoch 2
(ClientAppActor pid=2111) Train Loss: 0.1699
(ClientAppActor pid=2111) -----------mse_loss mean :  0.1332 std: 0.5324
(ClientAppActor pid=2111) Val: Accuracy: 0.9473  
(ClientAppActor pid=2111) Train : time 32.71 s Epoch 1
(ClientAppActor pid=2111) Train Loss: 0.3568
(ClientAppActor pid=2111) Train : time 30.26 s Epoch 2
(ClientAppActor pid=2111) Train Loss: 0.1229
(ClientAppActor pid=2111) -----------mse_loss mean :  0.0263 std: 0.2658
(ClientAppActor pid=2111) Val: Accuracy: 0.9926  
(ClientAppActor pid=2111) Train

...

model AE is saved in fed_best_models/FED_AVG_AE_f1_0.93_recall_1.00_.pth
(ClientAppActor pid=2111) Train : time 38.85 s Epoch 1
(ClientAppActor pid=2111) Train Loss: 0.0019
(ClientAppActor pid=2111) Train : time 37.84 s Epoch 2
(ClientAppActor pid=2111) Train Loss: 0.0018
(ClientAppActor pid=2111) -----------mse_loss mean :  0.0029 std: 0.0663
(ClientAppActor pid=2111) Val: Accuracy: 0.9971  
(ClientAppActor pid=2111) Train : time 28.81 s Epoch 1
(ClientAppActor pid=2111) Train Loss: 0.0028
(ClientAppActor pid=2111) Train : time 27.76 s Epoch 2
(ClientAppActor pid=2111) Train Loss: 0.0025
(ClientAppActor pid=2111) -----------mse_loss mean :  0.0024 std: 0.0333
(ClientAppActor pid=2111) Val: Accuracy: 0.9943  
(ClientAppActor pid=2111) Train : time 33.52 s Epoch 1
(ClientAppActor pid=2111) Train Loss: 0.0016
(ClientAppActor pid=2111) Train : time 32.10 s Epoch 2
(ClientAppActor pid=2111) Train Loss: 0.0016
(ClientAppActor pid=2111) -----------mse_loss mean :  0.0018 std: 0.0434
(ClientA

INFO :      aggregate_fit: received 4 results and 0 failures


Round 10: Aggregated threshold = 0.0420
(ClientAppActor pid=2111) -----------mse_loss mean :  0.0015 std: 0.0162
(ClientAppActor pid=2111) Val: Accuracy: 0.9968  
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 102697, 1: 36353})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 93, 1: 5260, 2: 14, 3: 15, 4: 4, 5: 1, 6: 10}
{0: 'threshold=0.0420 ,Test : Accuracy: 0.9612 Recall : 0.8724 FDR: 0.0026  F1-score: 0.9307 '}


INFO :      fit progress: (10, 0.11462859021035599, {0: 'threshold=0.0420 ,Test : Accuracy: 0.9612 Recall : 0.8724 FDR: 0.0026  F1-score: 0.9307 '}, 3009.08629122)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 3009.09s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.23868932038835
INFO :      		round 1: 0.6315283396260338
INFO :      		round 2: 0.6584192961165048
INFO :      		round 3: 0.6106793869111831
INFO :      		round 4: 0.5863155115066523
INFO :      		round 5: 0.5814916509349155
INFO :      		round 6: 0.5616749820208559
INFO :      		round 7: 0.5335463300071916
INFO :      		round 8: 0.11932899867403812
INFO :      		round 9: 0.09691114088906869
INFO :      		round 10: 0.11462859021035599
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  '
INFO :      

Federated learning simulation finished.


In [15]:

# Instantiate the configuration
cfg.STRATEGY="FED_PROX"
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


Using FedProx strategy with AE model.
Starting federated learning simulation...


2025-07-28 01:17:54,314	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 3645942990.0, 'object_store_memory': 1822971494.0, 'GPU': 1.0, 'node:127.0.1.1': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({1: 139050})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 97486}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}


INFO :      initial parameters (loss, other metrics): 17.46371988493348, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

(ClientAppActor pid=11854) Train : time 61.98 s Epoch 1
(ClientAppActor pid=11854) Train Loss: 0.0078
(ClientAppActor pid=11854) Train : time 63.10 s Epoch 2
(ClientAppActor pid=11854) Train Loss: 0.0077
(ClientAppActor pid=11854) -----------mse_loss mean :  0.0079 std: 0.1141
(ClientAppActor pid=11854) Val: Accuracy: 0.9922  
(ClientAppActor pid=11854) Train : time 72.20 s Epoch 1
(ClientAppActor pid=11854) Train Loss: 0.0087
(ClientAppActor pid=11854) Train : time 71.94 s Epoch 2
(ClientAppActor pid=11854) Train Loss: 0.0086
(ClientAppActor pid=11854) -----------mse_loss mean :  0.0066 std: 0.0758
(ClientAppActor pid=11854) Val: Accuracy: 0.9944  
(ClientAppActor pid=11854) Train : time 72.72 s Epoch 1
(ClientAppActor pid=11854) Train Loss: 0.0082
(ClientAppActor pid=11854) Train : time 72.76 s Epoch 2
(ClientAppActor pid=11854) Train Loss: 0.0081
(ClientAppActor pid=11854) -----------mse_loss mean :  0.0096 std: 0.1123
(ClientAppActor pid=11854) Val: Accuracy: 0.9913  
(ClientAppAct

INFO :      aggregate_fit: received 4 results and 0 failures


Round 10: Aggregated threshold = 0.1076
(ClientAppActor pid=11854) -----------mse_loss mean :  0.0103 std: 0.0939
(ClientAppActor pid=11854) Val: Accuracy: 0.9925  
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 102696, 1: 36354})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 94, 1: 5260, 2: 14, 3: 15, 4: 4, 5: 1, 6: 10}
{0: 'threshold=0.1076 ,Test : Accuracy: 0.9612 Recall : 0.8724 FDR: 0.0026  F1-score: 0.9307 '}


INFO :      fit progress: (10, 0.6762024114527149, {0: 'threshold=0.1076 ,Test : Accuracy: 0.9612 Recall : 0.8724 FDR: 0.0026  F1-score: 0.9307 '}, 5500.476163033)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 5500.48s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.46371988493348
INFO :      		round 1: 1.6673866190219346
INFO :      		round 2: 0.6073619538834951
INFO :      		round 3: 0.6329811106616325
INFO :      		round 4: 0.6579193635382956
INFO :      		round 5: 0.6744806274721323
INFO :      		round 6: 0.6658450197770586
INFO :      		round 7: 0.6674383652463143
INFO :      		round 8: 0.6694232065803668
INFO :      		round 9: 0.6787446624415678
INFO :      		round 10: 0.6762024114527149
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  '
INFO :      	  

Federated learning simulation finished.


### VAE

In [16]:
# Instantiate the configuration
cfg.STRATEGY="FED_AVG"
cfg.MODEL_NAME="VAE"
cfg.LEARNING_RATE=1e-4
cfg.WEIGHT_DECAY=1e-4
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 4},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


Using FedAvg strategy with VAE model.
Starting federated learning simulation...


2025-07-28 02:49:46,088	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 3636790887.0, 'object_store_memory': 1818395443.0, 'GPU': 1.0, 'node:127.0.1.1': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({1: 139050})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 97486}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}


INFO :      initial parameters (loss, other metrics): 17.469951456310678, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

(ClientAppActor pid=29203) Train : time 30.42 s Epoch 1
(ClientAppActor pid=29203) Train Loss: 0.1589
(ClientAppActor pid=29203) Train : time 30.35 s Epoch 2
(ClientAppActor pid=29203) Train Loss: 0.1580
(ClientAppActor pid=29203) -----------mse_loss mean :  0.0422 std: 0.1994
(ClientAppActor pid=29203) Val: Accuracy: 0.9840  
(ClientAppActor pid=29203) Train : time 39.79 s Epoch 1
(ClientAppActor pid=29203) Train Loss: 0.1593
(ClientAppActor pid=29203) Train : time 40.53 s Epoch 2
(ClientAppActor pid=29203) Train Loss: 0.1577
(ClientAppActor pid=29203) -----------mse_loss mean :  0.0396 std: 0.1766
(ClientAppActor pid=29203) Val: Accuracy: 0.9818  
(ClientAppActor pid=29203) Train : time 35.25 s Epoch 1
(ClientAppActor pid=29203) Train Loss: 0.1569
(ClientAppActor pid=29203) Train : time 35.12 s Epoch 2
(ClientAppActor pid=29203) Train Loss: 0.1556
(ClientAppActor pid=29203) -----------mse_loss mean :  0.0416 std: 0.2041
(ClientAppActor pid=29203) Val: Accuracy: 0.9836  
(ClientAppAct

INFO :      aggregate_fit: received 4 results and 0 failures


Round 10: Aggregated threshold = 0.2365
(ClientAppActor pid=29203) -----------mse_loss mean :  0.0425 std: 0.2001
(ClientAppActor pid=29203) Val: Accuracy: 0.9824  
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 101932, 1: 37118})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 817, 1: 5219, 2: 14, 3: 15, 4: 4, 5: 1, 6: 10}
{0: 'threshold=0.2365 ,Test : Accuracy: 0.9563 Recall : 0.8734 FDR: 0.0220  F1-score: 0.9227 '}


INFO :      fit progress: (10, 0.6100392170082704, {0: 'threshold=0.2365 ,Test : Accuracy: 0.9563 Recall : 0.8734 FDR: 0.0220  F1-score: 0.9227 '}, 3172.1540700660007)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 3172.16s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.469951456310678
INFO :      		round 1: 0.9280447680690399
INFO :      		round 2: 0.7108502449658396
INFO :      		round 3: 0.6718879786947142
INFO :      		round 4: 0.66302611470694
INFO :      		round 5: 0.6592584164868752
INFO :      		round 6: 0.6493307263574254
INFO :      		round 7: 0.6398248718985976
INFO :      		round 8: 0.641309331175836
INFO :      		round 9: 0.628583861470694
INFO :      		round 10: 0.6100392170082704
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  '
INFO :      	 

Federated learning simulation finished.


In [17]:
cfg.STRATEGY="FED_PROX"
strategy=set_server_strategy()

print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


Using FedProx strategy with VAE model.
Starting federated learning simulation...


2025-07-28 03:42:47,868	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 3587486516.0, 'object_store_memory': 1793743257.0, 'GPU': 1.0, 'node:127.0.1.1': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({1: 139050})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 97486}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}


INFO :      initial parameters (loss, other metrics): 17.594077669902912, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

model VAE is saved in fed_best_models/FED_PROX_VAE_f1_0.92_recall_1.00_.pth
(ClientAppActor pid=39413) Train : time 65.19 s Epoch 1
(ClientAppActor pid=39413) Train Loss: 0.1718
(ClientAppActor pid=39413) Train : time 64.56 s Epoch 2
(ClientAppActor pid=39413) Train Loss: 0.1716
(ClientAppActor pid=39413) -----------mse_loss mean :  0.0615 std: 0.4826
(ClientAppActor pid=39413) Val: Accuracy: 0.9925  
(ClientAppActor pid=39413) Train : time 64.04 s Epoch 1
(ClientAppActor pid=39413) Train Loss: 0.1727
(ClientAppActor pid=39413) Train : time 63.94 s Epoch 2
(ClientAppActor pid=39413) Train Loss: 0.1722
(ClientAppActor pid=39413) -----------mse_loss mean :  0.0616 std: 0.5035
(ClientAppActor pid=39413) Val: Accuracy: 0.9935  
(ClientAppActor pid=39413) Train : time 49.02 s Epoch 1
(ClientAppActor pid=39413) Train Loss: 0.1737
(ClientAppActor pid=39413) Train : time 49.48 s Epoch 2
(ClientAppActor pid=39413) Train Loss: 0.1731
(ClientAppActor pid=39413) -----------mse_loss mean :  0.0571 

INFO :      aggregate_fit: received 4 results and 0 failures


Round 10: Aggregated threshold = 0.5383
(ClientAppActor pid=39413) -----------mse_loss mean :  0.0608 std: 0.4816
(ClientAppActor pid=39413) Val: Accuracy: 0.9931  
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 102837, 1: 36213})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 257, 1: 5538, 2: 14, 3: 16, 4: 4, 5: 1, 6: 10, 7: 25}
{0: 'threshold=0.5383 ,Test : Accuracy: 0.9578 Recall : 0.8651 FDR: 0.0071  F1-score: 0.9246 '}


INFO :      fit progress: (10, 0.7513695613088817, {0: 'threshold=0.5383 ,Test : Accuracy: 0.9578 Recall : 0.8651 FDR: 0.0071  F1-score: 0.9246 '}, 4952.967729497999)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 4952.97s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.594077669902912
INFO :      		round 1: 0.6033506944444444
INFO :      		round 2: 0.6866325062927005
INFO :      		round 3: 0.690279575692197
INFO :      		round 4: 0.6981853986875225
INFO :      		round 5: 0.711616325062927
INFO :      		round 6: 0.7227742943185904
INFO :      		round 7: 0.7329697051420352
INFO :      		round 8: 0.7403540205861201
INFO :      		round 9: 0.7450732650125854
INFO :      		round 10: 0.7513695613088817
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  '
INFO :      	

Federated learning simulation finished.


In [18]:
# Instantiate the configuration
cfg.STRATEGY="FED_AVG"
cfg.MODEL_NAME="AAE"
cfg.LEARNING_RATE=1e-2
cfg.WEIGHT_DECAY=1e-5
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


Using FedAvg strategy with AAE model.
Starting federated learning simulation...


2025-07-28 05:05:32,141	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'object_store_memory': 1780974796.0, 'memory': 3561949595.0, 'GPU': 1.0, 'node:127.0.1.1': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({1: 139050})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 97486}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}


INFO :      initial parameters (loss, other metrics): 17.88722761596548, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

(ClientAppActor pid=55041) Train : time 66.91 s Epoch 1
(ClientAppActor pid=55041) Generator Loss: 0.3045 Discriminator Loss: 28.6363
(ClientAppActor pid=55041) Train : time 67.35 s Epoch 2
(ClientAppActor pid=55041) Generator Loss: 0.3532 Discriminator Loss: 25.9419
(ClientAppActor pid=55041) -----------mse_loss mean :  0.0057 std: 0.1218
(ClientAppActor pid=55041) Val: Accuracy: 0.9971  
(ClientAppActor pid=55041) Train : time 50.90 s Epoch 1
(ClientAppActor pid=55041) Generator Loss: 0.3648 Discriminator Loss: 28.2257
(ClientAppActor pid=55041) Train : time 50.65 s Epoch 2
(ClientAppActor pid=55041) Generator Loss: 0.3617 Discriminator Loss: 30.6840
(ClientAppActor pid=55041) -----------mse_loss mean :  0.0023 std: 0.0363
(ClientAppActor pid=55041) Val: Accuracy: 0.9932  
(ClientAppActor pid=55041) Train : time 66.86 s Epoch 1
(ClientAppActor pid=55041) Generator Loss: 0.3220 Discriminator Loss: 31.3597
(ClientAppActor pid=55041) Train : time 66.65 s Epoch 2
(ClientAppActor pid=5504

INFO :      aggregate_fit: received 4 results and 0 failures


Round 10: Aggregated threshold = 0.0709
(ClientAppActor pid=55041) -----------mse_loss mean :  0.0050 std: 0.0858
(ClientAppActor pid=55041) Val: Accuracy: 0.9941  
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 102751, 1: 36299})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 39, 1: 5260, 2: 14, 3: 15, 4: 4, 5: 1, 6: 10}
{0: 'threshold=0.0709 ,Test : Accuracy: 0.9616 Recall : 0.8724 FDR: 0.0011  F1-score: 0.9314 '}


INFO :      fit progress: (10, 0.2942409430061129, {0: 'threshold=0.0709 ,Test : Accuracy: 0.9616 Recall : 0.8724 FDR: 0.0011  F1-score: 0.9314 '}, 5089.477691605)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 5089.48s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.88722761596548
INFO :      		round 1: 0.8161512832614167
INFO :      		round 2: 1.2245423184106436
INFO :      		round 3: 0.23782384933477166
INFO :      		round 4: 0.23243985414419274
INFO :      		round 5: 0.33499820208558073
INFO :      		round 6: 0.305005365650845
INFO :      		round 7: 0.567540958737864
INFO :      		round 8: 0.2790209794138799
INFO :      		round 9: 0.3494181499460626
INFO :      		round 10: 0.2942409430061129
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  '
INFO :      	 

Federated learning simulation finished.


In [19]:
# Instantiate the configuration
cfg.STRATEGY="FED_PROX"
cfg.MODEL_NAME="AAE"
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


Using FedProx strategy with AAE model.
Starting federated learning simulation...


2025-07-28 06:30:32,075	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 3504940647.0, 'object_store_memory': 1752470323.0, 'GPU': 1.0, 'node:127.0.1.1': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({1: 139050})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 97486}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}


INFO :      initial parameters (loss, other metrics): 18.29554836389788, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  F1-score: 0.4603 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

(ClientAppActor pid=71132) Train : time 112.12 s Epoch 1
(ClientAppActor pid=71132) Generator Loss: 1.1799 Discriminator Loss: 24.4235
(ClientAppActor pid=71132) Train : time 117.29 s Epoch 2
(ClientAppActor pid=71132) Generator Loss: 1.1849 Discriminator Loss: 26.0010
(ClientAppActor pid=71132) -----------mse_loss mean :  0.0155 std: 0.2050
(ClientAppActor pid=71132) Val: Accuracy: 0.9946  
(ClientAppActor pid=71132) Train : time 102.16 s Epoch 1
(ClientAppActor pid=71132) Generator Loss: 1.3832 Discriminator Loss: 26.8476
(ClientAppActor pid=71132) Train : time 101.88 s Epoch 2
(ClientAppActor pid=71132) Generator Loss: 1.2983 Discriminator Loss: 24.6750
(ClientAppActor pid=71132) -----------mse_loss mean :  0.0237 std: 0.2351
(ClientAppActor pid=71132) Val: Accuracy: 0.9923  
(ClientAppActor pid=71132) Train : time 120.86 s Epoch 1
(ClientAppActor pid=71132) Generator Loss: 1.2578 Discriminator Loss: 24.0398
(ClientAppActor pid=71132) Train : time 112.52 s Epoch 2
(ClientAppActor pi

INFO :      aggregate_fit: received 4 results and 0 failures


Round 10: Aggregated threshold = 0.2437
(ClientAppActor pid=71132) -----------mse_loss mean :  0.0159 std: 0.1990
(ClientAppActor pid=71132) Val: Accuracy: 0.9926  
Counts of : original binary labels Counter({0: 97486, 1: 41564}) predicted binary labels Counter({0: 102696, 1: 36354})
Counts of  original  labels: {0: 97486, 1: 41306, 2: 59, 3: 40, 4: 55, 5: 40, 6: 36, 7: 28}
Counts of misclassified original labels: {0: 94, 1: 5260, 2: 14, 3: 15, 4: 4, 5: 1, 6: 10}
{0: 'threshold=0.2437 ,Test : Accuracy: 0.9612 Recall : 0.8724 FDR: 0.0026  F1-score: 0.9307 '}


INFO :      fit progress: (10, 0.10279007523148148, {0: 'threshold=0.2437 ,Test : Accuracy: 0.9612 Recall : 0.8724 FDR: 0.0026  F1-score: 0.9307 '}, 8188.509897224998)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 8188.51s
INFO :      	History (loss, centralized):
INFO :      		round 0: 18.29554836389788
INFO :      		round 1: 0.860093660104279
INFO :      		round 2: 0.3420018091513844
INFO :      		round 3: 1.0018960131247752
INFO :      		round 4: 0.3067594839985617
INFO :      		round 5: 0.5119737167385833
INFO :      		round 6: 0.41904328478964403
INFO :      		round 7: 0.3283092581355627
INFO :      		round 8: 0.20179080703883495
INFO :      		round 9: 0.20621707569219705
INFO :      		round 10: 0.10279007523148148
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2989 Recall : 1.0000 FDR: 0.7011  '
INFO :  

Federated learning simulation finished.


#### FedAVG evaluation

In [37]:
[col for sublist in SERVER_EVALUATION_DATA_MAPPING[:-1] for col in sublist]


['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-15-labeled.csv',
 './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-30-labeled.csv',
 './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-23-labeled.csv',
 './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-18-labeled.csv']

In [75]:
dataset_directory = "./ModbusDataset/" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

Trained_AE_model=AE(input_dim=76)
Trained_AE_model.load_state_dict(torch.load("./fed_best_models/FED_AVG_AE_f1_0.93_recall_1.00_.pth"))
Trained_VAE_model=VAE(input_dim=76)
Trained_VAE_model.load_state_dict(torch.load("./fed_best_models/FED_AVG_VAE_f1_0.92_recall_1.00_.pth"))
Trained_AAE_model=AdversarialAutoencoder()
Trained_AAE_model.load_state_dict(torch.load("./fed_best_models/FED_AVG_AdversarialAutoencoder_f1_0.93_recall_1.00_.pth"))
val_files=[col for sublist in SERVER_EVALUATION_DATA_MAPPING[:-1] for col in sublist]
# val_files=['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-26-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-23-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-21-labeled.csv']
model_thresholds={0:(Trained_AE_model,0.0412),1:(Trained_VAE_model,0.4293),2:(Trained_AAE_model, 0.0666)}


 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

In [ ]:
### test on total attack captures with Valid dataset of the clients
# we know that K=3 is better for AAE than K=1 (from centralized part)
test_files=[col for col in modbus.dataset["attack_dataset_dir"]["external"] if col.find("network-wide")!=-1]
test_files.extend( [col for col in modbus.dataset["attack_dataset_dir"]["compromised-ied"] if col.find("trust-scada-hmi")!=-1])
test_files.extend([col for col in modbus.dataset["attack_dataset_dir"]["compromised-scada"] if col.find("substation-wide-capture")!=-1])
print("test_files length",len(test_files))
print("benign valid files:",len(val_files))

val_dataloader=DataLoader(ModbusFlowStream(
            shuffle=False,
            chunk_size=1,
            batch_size=64,
            csv_files=val_files,
            scalers=loaded_scalers['network-wide']['min_max_scalers'],
        ),batch_size=1,shuffle=False)
test_dataloader=DataLoader(ModbusFlowStream(
            shuffle=False,
            chunk_size=1,
            batch_size=64,
            csv_files=test_files,
            scalers=loaded_scalers['network-wide']['min_max_scalers'],
        ),batch_size=1,shuffle=False)
for i,trained_model in model_thresholds.items():
    print("*"*10,trained_model[0]._get_name(),10*"*")
    train_eval(trained_model[0],None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0],k_range=[0,1,3,5])
    

test_files length 40
benign valid files: 4
********** AE **********

==================  lr=0, wd=0 ==================
--- Running Evaluation for Epoch 1 lr =0 wd 0 ---
-----------mse_loss mean :  0.0024 std: 0.0430
Val: Accuracy: 0.8293  
 K: 0 K-SIGMA Threshold : ---thr 0.002436
Counts of : original binary labels Counter({0: 4811517, 1: 396525}) predicted binary labels Counter({0: 3860664, 1: 1347378})
Counts of  original  labels: {0: 4811517, 1: 393454, 2: 611, 3: 507, 4: 505, 5: 510, 6: 425, 7: 470, 8: 43}
Counts of misclassified original labels: {0: 955133, 1: 4054, 2: 51, 3: 83, 5: 2, 6: 51, 7: 39}
Test : Accuracy: 0.8158 Recall : 0.9892 FDR: 0.7089  F1-score: 0.4498  
 K: 1 K-SIGMA Threshold : ---thr 0.04539
Counts of : original binary labels Counter({0: 4811517, 1: 396525}) predicted binary labels Counter({0: 4685639, 1: 522403})
Counts of  original  labels: {0: 4811517, 1: 393454, 2: 611, 3: 507, 4: 505, 5: 510, 6: 425, 7: 470, 8: 43}
Counts of misclassified original labels: {

In [ ]:
for scenario in {"compromised-ied","external","compromised-scada"}:
    if scenario=="compromised-scada":
        print("scenario :",scenario)
        dataset_directory = "./ModbusDataset" 
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("ied1b")!=-1]
        ### missed attack logs for the day 21 for ied1b which can reduce the accuracy.
        test_files.remove(dataset_directory+"/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")    
        # test_files.remove(dataset_directory+"/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")    

    elif scenario=="compromised-ied":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("trust-scada-hmi")!=-1]
    else:
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("network-wide")!=-1]        

    print("----------- benign valid files:",len(val_files),val_files)
    print(f"----------{scenario} attack  test files : ",len(test_files),test_files)
    val_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=1,
                batch_size=64,
                csv_files=val_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    test_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=1,
                batch_size=64,
                csv_files=test_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    for trained_model in {Trained_AE_model,Trained_VAE_model,Trained_AAE_model}:
        print("*"*10,trained_model._get_name(),10*"*")
        train_eval(trained_model,None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0])
        

scenario : compromised-ied
----------- benign valid files: 4 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-15-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-30-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-23-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-18-labeled.csv']
----------compromised-ied attack  test files :  20 ['./ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-2-labeled.csv', './ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-10-labeled.csv', './ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-19-labeled.csv', './ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-

### test Fedprox

In [80]:


Trained_AE_model=AE(input_dim=76)
Trained_AE_model.load_state_dict(torch.load("./fed_best_models/FED_PROX_AE_f1_0.93_recall_1.00_.pth"))
Trained_VAE_model=VAE(input_dim=76)
Trained_VAE_model.load_state_dict(torch.load("./fed_best_models/FED_PROX_VAE_f1_0.92_recall_1.00_.pth"))
Trained_AAE_model=AdversarialAutoencoder()
Trained_AAE_model.load_state_dict(torch.load("./fed_best_models/FED_PROX_AdversarialAutoencoder_f1_0.93_recall_1.00_.pth"))
# val_files=['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-26-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-23-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-21-labeled.csv']
model_thresholds={0:(Trained_AE_model,0.0412),1:(Trained_VAE_model,0.4293),2:(Trained_AAE_model, 0.0666)}


In [81]:
### test on total attack captures with Valid dataset of the clients
# we know that K=3 is better for AAE than K=1 (from centralized part)
test_files=[col for col in modbus.dataset["attack_dataset_dir"]["external"] if col.find("network-wide")!=-1]
test_files.extend( [col for col in modbus.dataset["attack_dataset_dir"]["compromised-ied"] if col.find("trust-scada-hmi")!=-1])
test_files.extend([col for col in modbus.dataset["attack_dataset_dir"]["compromised-scada"] if col.find("substation-wide-capture")!=-1])
print("test_files length",len(test_files))
print("benign valid files:",len(val_files))

val_dataloader=DataLoader(ModbusFlowStream(
            shuffle=False,
            chunk_size=1,
            batch_size=64,
            csv_files=val_files,
            scalers=loaded_scalers['network-wide']['min_max_scalers'],
        ),batch_size=1,shuffle=False)
test_dataloader=DataLoader(ModbusFlowStream(
            shuffle=False,
            chunk_size=1,
            batch_size=64,
            csv_files=test_files,
            scalers=loaded_scalers['network-wide']['min_max_scalers'],
        ),batch_size=1,shuffle=False)
for i,trained_model in model_thresholds.items():
    print("*"*10,trained_model[0]._get_name(),10*"*")
    train_eval(trained_model[0],None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0],k_range=[0,1,3,5])
    

test_files length 40
benign valid files: 4
********** AE **********

==================  lr=0, wd=0 ==================
--- Running Evaluation for Epoch 1 lr =0 wd 0 ---
-----------mse_loss mean :  0.0263 std: 0.2246
Val: Accuracy: 0.9316  
 K: 0 K-SIGMA Threshold : ---thr 0.02627
Counts of : original binary labels Counter({0: 4811517, 1: 396525}) predicted binary labels Counter({0: 4275045, 1: 932997})
Counts of  original  labels: {0: 4811517, 1: 393454, 2: 611, 3: 507, 4: 505, 5: 510, 6: 425, 7: 470, 8: 43}
Counts of misclassified original labels: {0: 572274, 1: 35397, 2: 97, 3: 164, 4: 4, 5: 2, 6: 90, 7: 47, 8: 1}
Test : Accuracy: 0.8832 Recall : 0.9097 FDR: 0.6134  F1-score: 0.5426  
 K: 1 K-SIGMA Threshold : ---thr 0.2509
Counts of : original binary labels Counter({0: 4811517, 1: 396525}) predicted binary labels Counter({0: 4671845, 1: 536197})
Counts of  original  labels: {0: 4811517, 1: 393454, 2: 611, 3: 507, 4: 505, 5: 510, 6: 425, 7: 470, 8: 43}
Counts of misclassified origina

In [82]:
for scenario in {"compromised-ied","external","compromised-scada"}:
    if scenario=="compromised-scada":
        print("scenario :",scenario)
        dataset_directory = "./ModbusDataset" 
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("ied1b")!=-1]
        ### missed attack logs for the day 21 for ied1b which can reduce the accuracy.
        test_files.remove(dataset_directory+"/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")    

    elif scenario=="compromised-ied":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("trust-scada-hmi")!=-1]
    else:
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("network-wide")!=-1]        

    print("----------- benign valid files:",len(val_files),val_files)
    print(f"----------{scenario} attack  test files : ",len(test_files),test_files)
    val_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=1,
                batch_size=64,
                csv_files=val_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    test_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=1,
                batch_size=64,
                csv_files=test_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    for trained_model in {Trained_AE_model,Trained_VAE_model,Trained_AAE_model}:
        print("*"*10,trained_model._get_name(),10*"*")
        train_eval(trained_model,None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0])
        

scenario : compromised-ied
----------- benign valid files: 4 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-15-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-30-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-23-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-18-labeled.csv']
----------compromised-ied attack  test files :  20 ['./ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-2-labeled.csv', './ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-10-labeled.csv', './ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-19-labeled.csv', './ModbusDataset/attack/compromised-ied/trust-scada-hmi/trust-scada-network-captures/ready/veth3efd353-